In [ ]:
!nvidia-smi

Wed Apr 21 19:19:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore sentencepiece
    !pip install -Uqq --no-deps fastai
    !pip install -Uqq transformers datasets wandb 

     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 1.2MB 9.7MB/s 
     |████████████████████████████████| 194kB 9.1MB/s 
     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 204kB 55.0MB/s 
     |████████████████████████████████| 2.1MB 52.3MB/s 
     |████████████████████████████████| 3.3MB 52.0MB/s 
     |████████████████████████████████| 901kB 54.5MB/s 
     |████████████████████████████████| 245kB 57.9MB/s 
     |████████████████████████████████| 112kB 60.4MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 174kB 54.8MB/s 
     |████████████████████████████████| 133kB 59.4MB/s 
     |████████████████████████████████| 71kB 12.6MB/s 


In [ ]:
from transformers import *
from datasets import load_dataset, concatenate_datasets, load_metric

## Setup

In [ ]:
model_name = 'distilroberta-base'
# data
max_length = 512
bs = 64
val_bs = bs*2
# training
lr = 3e-5

## Data preprocessing

In [ ]:
ds_name = 'snli'

In [ ]:
train_ds = load_dataset(ds_name, split='train')
valid_ds = load_dataset(ds_name, split='validation')


Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


Reusing dataset snli (/root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


In [ ]:
# train_ds = train_ds.select(range(100))
# valid_ds = valid_ds.select(range(100))

In [ ]:
len(train_ds), len(valid_ds)

(550152, 10000)

In [ ]:
train_ds.column_names

['premise', 'hypothesis', 'label']

In [ ]:
train_ds[2]

{'hypothesis': 'A person is outdoors, on a horse.',
 'label': 0,
 'premise': 'A person on a horse jumps over a broken down airplane.'}

In [ ]:
from collections import Counter

In [ ]:
Counter(train_ds['label'])

Counter({-1: 785, 0: 183416, 1: 182764, 2: 183187})

In [ ]:
train_ds = train_ds.filter(lambda sample: sample['label'] in [0,1,2])
valid_ds = valid_ds.filter(lambda sample: sample['label'] in [0,1,2])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def tokenize(batch):
    batch = tokenizer(
        batch['premise'],
        batch['hypothesis'],
        add_special_tokens=True,
        padding=False,
        truncation=True,
        max_length=max_length,
        # return_tensors='pt'
        )
    return batch

In [ ]:
train_ds = train_ds.map(tokenize, batched=True, batch_size=100, remove_columns=['premise','hypothesis'])
valid_ds = valid_ds.map(tokenize, batched=True, batch_size=100, remove_columns=['premise','hypothesis'])

In [ ]:
train_ds = train_ds.rename_column('label', 'labels')
valid_ds = valid_ds.rename_column('label', 'labels')

## Tracking

In [ ]:
import wandb

WANDB_NAME = f'{ds_name}-{model_name}-hf'
GROUP = f'{ds_name}-{model_name}-hf-{lr:.0e}'
NOTES = f'HF finetuning {model_name} with AdamW lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name,ds_name,'adamw']

In [ ]:
# import wandb

# WANDB_NAME = f'{ds_name}-{model_name}-alum'
# GROUP = f'{ds_name}-{model_name}-hf-{lr:.0e}'
# NOTES = f'HF finetuning {model_name} with RAdam lr={lr:.0e}'
# CONFIG = {}
# TAGS =[model_name,ds_name,'radam','alum']

In [ ]:
%env WANDB_LOG_MODEL = false
%env WANDB_WATCH = false

env: WANDB_LOG_MODEL=false
env: WANDB_WATCH=false


In [ ]:
wandb.init(reinit=True, project="vat", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

In [ ]:
training_args = TrainingArguments(
    'test-snli',
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = bs,
    per_device_eval_batch_size=val_bs,
    learning_rate=lr,
    num_train_epochs=5,
    warmup_ratio=0.2,
    logging_steps=200,
    fp16=True,
    group_by_length=True,
    report_to='wandb',
    save_strategy='epoch',
    save_total_limit=2,
    seed=8,
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [ ]:
metric = load_metric('accuracy')

In [ ]:
import numpy as np

## Regular training

In [ ]:
def compute_metric(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    # data_collator=DataCollatorWithPadding(),
    compute_metrics=compute_metric
)

In [ ]:
out = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.415900,0.345860,0.868015,15.654300,628.707000
2,0.338700,0.293040,0.891892,15.626200,629.841000
3,0.280900,0.277979,0.902662,15.651300,628.829000


In [ ]:
wandb.finish()

## VATrainer

In [ ]:
from core import compute_adversarial_loss

In [ ]:
class VATrainer(Trainer):

    def __init__(self, *args, vat_kwargs={}, **kwargs):
        super().__init__(*args, **kwargs)
        self.adv_alpha = vat_kwargs.pop('alpha', 1.)
        self.mask_special_tokens = vat_kwargs.pop('mask_special_tokens', False)
        self.one_token_type = vat_kwargs.pop('one_token_type', False)
        self.vat_start_epoch = vat_kwargs.pop('start_epoch', 1)
        self.vat_kwargs = vat_kwargs

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.

        Subclass and override for custom behavior.
        """
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        # explicitly adding kwargs here, verify no conflicts may happen
        outputs = model(**inputs, output_hidden_states=model.training, return_dict=True)
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            loss = outputs.loss
        #TODO add option to use vat_start_step
        if model.training and self.state.epoch >= self.vat_start_epoch:
            # ALUM training procedure
            embed = outputs.hidden_states[0].detach()
            # TODO add option mask special tokens or toke types here
            special_tokens_mask, token_type_mask = None, None
            if self.mask_special_tokens:
                special_tokens_mask = None
            if self.one_token_type:
                token_type_mask = None

            adv_loss = compute_adversarial_loss(model, embed, outputs.logits, 
                special_tokens_mask=special_tokens_mask, token_type_mask=token_type_mask,
                **self.vat_kwargs)
            loss += adv_loss
        return (loss, outputs) if return_outputs else loss

## Training

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.weight'

In [ ]:
trainer = VATrainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    # data_collator=DataCollatorWithPadding(),
    compute_metrics=compute_metric
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.286500,1.096642,0.363636,19.236800,5.146000
2,1.215700,1.096506,0.363636,18.539200,5.340000
3,1.198100,1.097012,0.363636,19.147600,5.170000
4,1.236400,1.096915,0.363636,18.583200,5.327000
5,1.221100,1.096867,0.363636,18.762900,5.276000


TrainOutput(global_step=20, training_loss=1.222323226928711, metrics={'train_runtime': 1292.5637, 'train_samples_per_second': 0.015, 'total_flos': 15169993962696.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 8192, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 376832, 'train_mem_cpu_peaked_delta': 0})

## Validation on adversarial data

In [ ]:
adv_ds = load_dataset('anli', split='test_r1')
adv_ds[0]

Reusing dataset anli (/home/.cache/huggingface/datasets/anli/plain_text/0.1.0/43fa2c99c10bf8478f1fa0860f7b122c6b277c4c41306255b7641257cf4e3299)


{'hypothesis': 'The first Ernest Jones store was opened on the continent of Europe.',
 'label': 0,
 'premise': 'Ernest Jones is a British jeweller and watchmaker. Established in 1949, its first store was opened in Oxford Street, London. Ernest Jones specialises in diamonds and watches, stocking brands such as Gucci and Emporio Armani. Ernest Jones is part of the Signet Jewelers group.',
 'reason': "The first store was opened in London, which is in Europe. It may have been difficult for the system because continents weren't mentioned.",
 'uid': '4aae63a8-fcf7-406c-a2f3-50c31c5934a9'}